In [ ]:
import torch
from recbole.config import Config
from recbole.data import create_dataset
from recbole.data.utils import data_preparation
from recbole.model.sequential_recommender.fearec import FEARec
from recbole.model.knowledge_aware_recommender.kgin import KGIN
from recbole.model.sequential_recommender.bert4rec import BERT4Rec
from recbole.model.sequential_recommender.sasrec import SASRec
from recbole.trainer import Trainer
from recbole.utils.enum_type import KGDataLoaderState
from recbole.data.interaction import Interaction
from recbole.quick_start import load_data_and_model

import gc

In [ ]:
import math
import numpy as np
from sklearn.metrics import ndcg_score


def recall_at_k(preds, targets, k=10):
    recall = []
    for pred, target in zip(preds, targets):
        recall.append(1.0 if target in pred[:k] else 0.0)
    return sum(recall) / len(recall)

def ndcg_at_k_ls(pred_ranking, target_item, k=10):
    """
    pred_ranking: list[int] các item theo thứ tự giảm dần điểm
    target_item : int, ground-truth item (LS)
    k           : cắt top-k
    """
    topk = pred_ranking[:k]
    if target_item in topk:
        rank = topk.index(target_item) + 1  # 1-based
        return 1.0 / math.log2(rank + 1)    # IDCG = 1 với 1 relevant
    return 0.0

def mean_ndcg_at_k_ls(all_preds, all_targets, k=10):
    """
    all_preds  : List[List[int]]
    all_targets: List[int]
    """
    s = 0.0
    n = len(all_targets)
    for pred, tgt in zip(all_preds, all_targets):
        s += ndcg_at_k_ls(pred, tgt, k)
    return s / n if n > 0 else 0.0


def mrr_at_k(preds, targets, k=10):
    reciprocal_ranks = []
    for pred, target in zip(preds, targets):
        if target in pred[:k]:
            rank = pred.index(target) + 1  # 1-based
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0)
    return sum(reciprocal_ranks) / len(reciprocal_ranks)

def precision_at_k(preds, targets, k=10):
    precision = []
    for pred, target in zip(preds, targets):
        precision.append(1.0 / k if target in pred[:k] else 0.0)
    return sum(precision) / len(precision)

def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    m = np.max(x)
    e = np.exp(x - m)
    s = e.sum()
    if s == 0:
        return np.full_like(x, 1.0 / len(x))
    return e / s


# MovieLens_1m

## SASRec

In [ ]:
sasrec_params = {
    'dataset': 'ml-1m',
    'data_path': '/content/drive/MyDrive/Recbole_data/',
    'field_separator': '\t',
    'seq_separator': ' ',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',

    'ITEM_LIST_LENGTH_FIELD': 'item_length',
    'LIST_SUFFIX': '_list',
    'MAX_ITEM_LIST_LENGTH': 100,

    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp']
    },
    'filter_inter_by_user_or_item': True,
    'user_inter_num_interval': "[10,inf]",
    'normalize_all': False,
    'repeatable': True,

    # ==== Model ====
    'hidden_size': 64,
    'n_layers': 2,
    'n_heads': 2,
    'inner_size': 256,
    'dropout_prob': 0.1,
    'hidden_dropout_prob': 0.1,
    'attn_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'layer_norm_eps': 1e-12,
    'initializer_range': 0.02,

    'loss_type': 'CE',
    'train_neg_sample_args': None,
    'epochs': 50,
    'train_batch_size': 512,
    'eval_batch_size': 1024,
    'learner': 'adam',
    'learning_rate': 0.0005,
    'eval_step': 2,
    'stopping_step': 10,

    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
    'topk': [10],
    'valid_metric': 'Recall@10',
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'
    },

    'seed': 2020,
    'device': 'cuda',
    'reproducibility': True
}

In [ ]:
sasrec_config = Config(model='SASRec', dataset='ml-1m', config_dict=sasrec_params)
sasrec_dataset = create_dataset(sasrec_config)
sasrec_train_data, sasrec_valid_data, sasrec_test_data = data_preparation(sasrec_config, sasrec_dataset)
sasrec_device = sasrec_config['device']
sasrec_model = SASRec(sasrec_config, sasrec_dataset).to(sasrec_device)
sasrec_model.load_state_dict(torch.load('SASRec_ML1M_ls.pth'))

# evaluate seq_model
for k in [5, 10, 20, 50]:
    sasrec_model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in sasrec_test_data:
        scores = sasrec_model.full_sort_predict(batch[0].to(sasrec_device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
kgin_params = {
    'data_path': '/content/drive/MyDrive/Recbole_data/',
    'seed': 2020,
    'reproducibility': True,

    'device': 'cuda',

    'epochs': 50,
    'train_batch_size': 512,
    'eval_batch_size': 1024,
    'learning_rate': 0.001,
    'embedding_size': 64,
    'loss_type': 'BPR',
    'train_neg_sample_args': {
        'sample_num': 1,
        'strategy': 'uniform'
    },

    'stopping_step': 10,
    'eval_step': 2,
    'validation_metric': 'Recall@10',
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'order': 'TO',
        'group_by': 'user',
        'mode': 'full'
    },
    'log_wandb': False,

    'field_separator': '\t',
    'seq_separator': ' ',

    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',
    'NEG_PREFIX': 'neg_',

    'HEAD_ENTITY_ID_FIELD': 'head_id',
    'TAIL_ENTITY_ID_FIELD': 'tail_id',
    'RELATION_ID_FIELD': 'relation_id',
    'ENTITY_ID_FIELD': 'entity_id',

    'alias_of_user_id': ['user_id'],
    'alias_of_item_id': ['item_id'],
    'alias_of_entity_id': ['head_id', 'tail_id'],
    'alias_of_relation_id': ['relation_id'],

    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp'],
        'kg': ['head_id', 'relation_id', 'tail_id'],
        'link': ['item_id', 'entity_id']
    },

    'filter_inter_by_user_or_item': True,
    'normalize_all': False
}

In [ ]:
kg_config = Config(model='KGIN', dataset='ml-1m', config_dict=kgin_params)
kg_dataset = create_dataset(kg_config)
kg_train_data, kg_valid_data, kg_test_data = data_preparation(kg_config, kg_dataset)
kg_device = kg_config['device']
kg_model = KGIN(kg_config, kg_dataset).to(kg_device)
kg_model.load_state_dict(torch.load('KGIN_ML1M_ls.pth'))

# Ma trận điểm kg_model đã padding

user_ids = kg_train_data._dataset['user_id'].tolist()
item_ids = kg_train_data._dataset['item_id'].tolist()

from collections import defaultdict

user_seen_items = defaultdict(set)

for u, i in zip(user_ids, item_ids):
    user_seen_items[u].add(i)

# compute KG-based scores
kgin_ml1m_scores = []

for user_id in range(kg_dataset.user_num):
    user_ids = [user_id] * kg_dataset.item_num
    item_ids = list(range(kg_dataset.item_num))

    interaction = Interaction({
        'user_id': torch.LongTensor(user_ids),
        'item_id': torch.LongTensor(item_ids)
    }).to(kg_device)

    with torch.no_grad():
        scores = kg_model.predict(interaction).cpu().tolist()

    min_score = min(scores)
    seen = user_seen_items[user_id]
    for j in seen:
        # scores[j] = float('-inf')
        scores[j] = min_score

    kgin_ml1m_scores.append(scores)

# compute KG-based scores mapping seq dataset
kgin_ml1m_scores_for_seq = []
for user_id in range(sasrec_dataset.user_num):
    scores = []
    for item_id in range(sasrec_dataset.item_num):
        item_token = sasrec_dataset.id2token('item_id', item_id)
        if item_token in kg_dataset.field2token_id['item_id']:
            kg_item_id = kg_dataset.token2id('item_id', item_token)
            scores.append(kgin_ml1m_scores[user_id][kg_item_id])
        else:
            scores.append(0.0)
    kgin_ml1m_scores_for_seq.append(scores)

kgin_ml1m_scores_for_seq = torch.tensor(kgin_ml1m_scores_for_seq).to(sasrec_device)

# del kg_model
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# evaluate hybrid scores
sasrec_model.eval()

for alpha in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    all_ranks = []
    all_ground_truths = []
    for batch in sasrec_test_data:
        scores = sasrec_model.full_sort_predict(batch[0].to(sasrec_device))
        scores = softmax_np(scores)
        for i, user_id in enumerate(batch[0]['user_id']):
            scores[i] = (1.0 - alpha) * scores[i] + alpha * kgin_ml1m_scores_for_seq[user_id]

        # Get top-K predicted items
        topk = scores.topk(k=10, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')


## BERT4Rec

In [ ]:
bert4rec_params = {
    'dataset': 'ml-1m',
    'data_path': '/content/drive/MyDrive/Recbole_data/',
    'field_separator': '\t',
    'seq_separator': ' ',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',

    'ITEM_LIST_LENGTH_FIELD': 'item_length',
    'LIST_SUFFIX': '_list',
    'MAX_ITEM_LIST_LENGTH': 100,

    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp']
    },
    'filter_inter_by_user_or_item': True,
    'user_inter_num_interval': "[10,inf]",
    'normalize_all': False,
    'repeatable': True,

    'hidden_size': 64,
    'n_layers': 2,
    'n_heads': 2,
    'inner_size': 256,
    'dropout_prob': 0.1,
    'hidden_dropout_prob': 0.1,
    'attn_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'mask_ratio': 0.2,
    'layer_norm_eps': 1e-12,
    'initializer_range': 0.02,

    'loss_type': 'CE',
    'train_neg_sample_args': None,
    'epochs': 50,
    'train_batch_size': 512,
    'eval_batch_size': 1024,
    'learner': 'adam',
    'learning_rate': 0.0005,
    'eval_step': 2,
    'stopping_step': 10,

    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
    'topk': [10],
    'valid_metric': 'Recall@10',
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'
    },

    'seed': 2020,
    'device': 'cuda',
    'reproducibility': True
}

In [ ]:
bert4rec_config = Config(model='BERT4Rec', dataset='ml-1m', config_dict=bert4rec_params)
bert4rec_dataset = create_dataset(bert4rec_config)
bert4rec_train_data, bert4rec_valid_data, bert4rec_test_data = data_preparation(bert4rec_config, bert4rec_dataset)
bert4rec_device = bert4rec_config['device']
bert4rec_model = BERT4Rec(bert4rec_config, bert4rec_dataset).to(bert4rec_device)
bert4rec_model.load_state_dict(torch.load('BERT4Rec_ML1M_ls.pth'))

# evaluate seq_model
for k in [5, 10, 20, 50]:
    bert4rec_model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in bert4rec_test_data:
        scores = bert4rec_model.full_sort_predict(batch[0].to(bert4rec_device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
# evaluate hybrid scores
bert4rec_model.eval()

for alpha in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    all_ranks = []
    all_ground_truths = []
    for batch in bert4rec_test_data:
        scores = bert4rec_model.full_sort_predict(batch[0].to(bert4rec_device))
        scores = softmax_np(scores)
        for i, user_id in enumerate(batch[0]['user_id']):
            scores[i] = (1.0 - alpha) * scores[i] + alpha * kgin_ml1m_scores_for_seq[user_id]

        # Get top-K predicted items
        topk = scores.topk(k=10, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')


## FEARec

In [ ]:
fearec_params = {
    'data_path': '/content/drive/MyDrive/Recbole_data/',
    'field_separator': '\t',
    'seq_separator': ' ',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',
    'NEG_PREFIX': 'neg_',

    'ITEM_LIST_LENGTH_FIELD': 'item_length',
    'LIST_SUFFIX': '_list',
    'MAX_ITEM_LIST_LENGTH': 50,

    'alias_of_user_id': ['user_id'],
    'alias_of_item_id': ['item_id'],

    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp']
    },
    'filter_inter_by_user_or_item': True,
    'user_inter_num_interval': "[10,inf]",
    'normalize_all': False,
    'repeatable': True,

    'device': 'cuda',
    'seed': 2020,
    'reproducibility': True,

    'epochs': 50,
    'train_batch_size': 256,
    'eval_batch_size': 512,
    'learning_rate': 0.001,
    'loss_type': 'CE',
    'train_neg_sample_args': None,
    'stopping_step': 10,
    'eval_step': 1,
    'validation_metric': 'Recall@10',

    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'order': 'TO',
        'mode': 'full'
    },
    'log_wandb': False,

    'embedding_size': 64,
    'hidden_size': 64,
    'n_heads': 2,
    'num_layers': 2,
    'dropout_prob': 0.1,

    'use_ssl': True,
    'use_supcon': True,
    'ssl_temp': 0.2,
    'supcon_weight': 0.1,
    'ssl_weight': 0.1,
}

In [ ]:
fearec_config = Config(model='FEARec', dataset='ml-1m', config_dict=fearec_params)
fearec_dataset = create_dataset(fearec_config)
fearec_train_data, fearec_valid_data, fearec_test_data = data_preparation(fearec_config, fearec_dataset)
fearec_device = fearec_config['device']
fearec_model = FEARec(fearec_config, fearec_dataset).to(fearec_device)
fearec_model.load_state_dict(torch.load('FEARec_ML1M_ls.pth'))

# evaluate seq_model
for k in [5, 10, 20, 50]:
    fearec_model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in fearec_test_data:
        scores = fearec_model.full_sort_predict(batch[0].to(fearec_device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
# evaluate hybrid scores
fearec_model.eval()

for alpha in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    all_ranks = []
    all_ground_truths = []
    for batch in fearec_test_data:
        scores = fearec_model.full_sort_predict(batch[0].to(fearec_device))
        scores = softmax_np(scores)
        for i, user_id in enumerate(batch[0]['user_id']):
            scores[i] = (1.0 - alpha) * scores[i] + alpha * kgin_ml1m_scores_for_seq[user_id]

        # Get top-K predicted items
        topk = scores.topk(k=10, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')


# Amazon_Books

## SASRec

In [ ]:
sasrec_config = {
    'data_path': 'D:\Personal\RecBole',
    'seed': 2020,
    'reproducibility': True,
    'device': 'cuda',

    # Training
    'epochs': 50,
    'train_batch_size': 256,
    'eval_batch_size': 512,
    'learning_rate': 0.001,
    'loss_type': 'CE',
    'train_neg_sample_args': None,  # Bắt buộc với CE

    # Evaluation
    'eval_step': 2,
    'stopping_step': 5,
    'validation_metric': 'Recall@10',
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'
    },

    'log_wandb': False,
    'save_checkpoint': True,

    # Data fields
    'field_separator': '\t',
    'seq_separator': ' ',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',
    'NEG_PREFIX': 'neg_',

    'alias_of_user_id': ['user_id'],
    'alias_of_item_id': ['item_id'],

    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp']
    },

    # Sequence-specific
    'normalize_all': False,
    'MAX_ITEM_LIST_LENGTH': 40,
    'ITEM_LIST_LENGTH_FIELD': 'item_length',
    'LIST_SUFFIX': '_list',
}


In [ ]:
config = Config(model='SASRec', dataset='Amazon_Books_filtered', config_dict=sasrec_config)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
device = config['device']
model = SASRec(config, dataset).to(device)

trainer = Trainer(config, model)
print("[DEBUG] Bắt đầu huấn luyện")
trainer.fit(train_data, valid_data)
res = trainer.evaluate(test_data)
print(res)
torch.save(model.state_dict(), 'SASRec_AB_filtered_ls.pth')

del model
gc.collect(); torch.cuda.empty_cache()

In [ ]:
kgin_config = {
    'data_path': 'D:\Personal\RecBole',
    'seed': 2020,
    'reproducibility': True,

    'device': 'cuda',

    'epochs': 50,
    'train_batch_size': 256,
    'eval_batch_size': 512,
    'learning_rate': 0.001,
    'embedding_size': 64,
    'loss_type': 'BPR',
    'train_neg_sample_args': {
        'sample_num': 1,
        'strategy': 'uniform'
    },

    'stopping_step': 5,
    'eval_step': 1,
    'validation_metric': 'Recall@10',
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'
    },
    'log_wandb': False,

    'field_separator': '\t',
    'seq_separator': ' ',

    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',
    'NEG_PREFIX': 'neg_',

    'HEAD_ENTITY_ID_FIELD': 'head_id',
    'TAIL_ENTITY_ID_FIELD': 'tail_id',
    'RELATION_ID_FIELD': 'relation_id',
    'ENTITY_ID_FIELD': 'entity_id',

    'alias_of_user_id': ['user_id'],
    'alias_of_item_id': ['item_id'],
    'alias_of_entity_id': ['head_id', 'tail_id'],
    'alias_of_relation_id': ['relation_id'],

    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp'],
        'kg': ['head_id', 'relation_id', 'tail_id'],
        'link': ['item_id', 'entity_id']
    },
    'normalize_all': False
}


In [ ]:
kg_config = Config(model='KGIN', dataset='Amazon_Books_filtered', config_dict=kgin_config)
kg_dataset = create_dataset(kg_config)
kg_train_data, kg_valid_data, kg_test_data = data_preparation(kg_config, kg_dataset)
kg_device = kg_config['device']
kg_model = KGIN(kg_config, kg_dataset).to(kg_device)


kg_trainer = Trainer(kg_config, kg_model)
kg_train_data.set_mode(KGDataLoaderState.RS)
kg_trainer.fit(kg_train_data, kg_valid_data)
kg_res = kg_trainer.evaluate(kg_test_data)
print(kg_res)
torch.save(kg_model.state_dict(), 'KGIN_AB_filtered_ls.pth')

del kg_model
gc.collect(); torch.cuda.empty_cache()

In [ ]:
sasrec_config = Config(model='SASRec', dataset='Amazon_Books_filtered', config_dict=sasrec_config)
sasrec_dataset = create_dataset(sasrec_config)
sasrec_train_data, sasrec_valid_data, sasrec_test_data = data_preparation(sasrec_config, sasrec_dataset)
sasrec_device = sasrec_config['device']
sasrec_model = SASRec(sasrec_config, sasrec_dataset).to(sasrec_device)
sasrec_model.load_state_dict(torch.load('SASRec_AB_filtered_ls.pth'))


In [ ]:
# evaluate seq_model
for k in [5, 10, 20, 50]:
    sasrec_model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in sasrec_test_data:
        scores = sasrec_model.full_sort_predict(batch[0].to(sasrec_device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
kg_config = Config(model='KGIN', dataset='Amazon_Books_filtered', config_dict=kgin_config)
kg_dataset = create_dataset(kg_config)
kg_train_data, kg_valid_data, kg_test_data = data_preparation(kg_config, kg_dataset)
kg_device = kg_config['device']
kg_model = KGIN(kg_config, kg_dataset).to(kg_device)
kg_model.load_state_dict(torch.load('KGIN_AB_filtered_ls.pth'))

In [ ]:
import numpy as np
import torch
from tqdm import tqdm
from recbole.data.interaction import Interaction

def build_kg_matrix_in_ram(
    sasrec_dataset, sasrec_test_data,
    kg_dataset, kg_model,
    dtype=np.float32,           # đổi sang np.float16 nếu cần tiết kiệm RAM
    show_progress=True
):
    """
    Trả về:
      - kg_matrix: np.ndarray shape [num_rows, n_items_seq], dtype=dtype
      - row_uid_tokens: list[str] thứ tự user_token tương ứng từng hàng
      - seq_pos: np.ndarray[int64] các cột SEQ có tương ứng trong KG
      - kg_ids : np.ndarray[int64] id item tương ứng bên KG (cùng chiều với seq_pos)
    """

    kg_model.eval()
    kg_device = next(kg_model.parameters()).device

    # === Không gian item SEQ và map sang KG ===
    n_items_seq = sasrec_dataset.item_num
    seq_tokens = [sasrec_dataset.id2token('item_id', i) for i in range(n_items_seq)]
    kg_tok2id_item = kg_dataset.field2token_id['item_id']  # token->id (KG)
    kg_idx_for_seq = np.array([kg_tok2id_item.get(t, -1) for t in seq_tokens], dtype=np.int64)

    # Giới hạn theo kích thước embedding hiện có của KG (an toàn CUDA)
    ue = getattr(kg_model, "user_embedding", None) or getattr(kg_model, "entity_embedding", None)
    ie = getattr(kg_model, "item_embedding", None) or getattr(kg_model, "relation_embedding", None)
    MAX_U = None if ue is None else ue.num_embeddings
    MAX_I = None if ie is None else ie.num_embeddings

    valid_mask = kg_idx_for_seq >= 0
    if MAX_I is not None:
        valid_mask &= (kg_idx_for_seq < MAX_I)

    seq_pos = np.flatnonzero(valid_mask)     # cột trong SEQ có tương ứng ở KG
    kg_ids  = kg_idx_for_seq[seq_pos]        # item_id tương ứng bên KG

    if len(seq_pos) == 0:
        raise RuntimeError("Không có item chung hợp lệ giữa SEQ và KG (seq_pos rỗng).")

    # ====== Duyệt test loader & ghép thành ma trận trong RAM ======
    rows = []
    row_uid_tokens = []

    it = tqdm(sasrec_test_data, desc="KG → RAM (per-row)", disable=not show_progress)
    with torch.no_grad():
        for batch in it:
            inp = batch[0]
            user_ids   = inp['user_id'].cpu().tolist()
            item_lists = inp['item_id_list'].cpu().tolist()   # index theo SEQ

            for i, uid in enumerate(user_ids):
                # Hàng kết quả (không gian SEQ), mặc định 0.0 cho item không có trong KG
                row = np.zeros(n_items_seq, dtype=dtype)

                uid_tok = sasrec_dataset.id2token('user_id', uid)
                row_uid_tokens.append(uid_tok)

                # Tính điểm KG chỉ trên item chung
                sub = None
                if (uid_tok in kg_dataset.field2token_id['user_id']) and (len(seq_pos) > 0):
                    kg_uid = kg_dataset.token2id('user_id', uid_tok)
                    if (MAX_U is None) or (kg_uid < MAX_U):
                        inter = Interaction({
                            'user_id': torch.full((len(kg_ids),), kg_uid, dtype=torch.long),
                            'item_id': torch.as_tensor(kg_ids, dtype=torch.long),
                        }).to(kg_device)

                        try:
                            out = kg_model.predict(inter)
                        except AttributeError:
                            out = kg_model.forward(inter)

                        sub = out.detach().cpu().numpy().reshape(-1).astype(dtype)  # len == len(kg_ids)
                        row[seq_pos] = sub

                # Gán min-score cho item đã tương tác (index theo SEQ), loại pad=0 nếu có
                if sub is not None and sub.size > 0:
                    min_score = dtype(sub.min())
                else:
                    min_score = dtype(0.0)

                seen_idx = np.fromiter((x for x in set(item_lists[i]) if 0 < x < n_items_seq), dtype=np.int64)
                if seen_idx.size > 0:
                    row[seen_idx] = min_score

                rows.append(row)

    # Ghép tất cả hàng thành 1 ma trận trong RAM
    kg_matrix = np.stack(rows, axis=0)
    return kg_matrix, row_uid_tokens, seq_pos, kg_ids

# ==== Ví dụ gọi hàm (giữ trong RAM) ====
# CẢNH BÁO RAM: cân nhắc đổi dtype=np.float16 nếu n_items_seq lớn
kg_matrix, row_uid_tokens, SEQ_POS, KG_IDS = build_kg_matrix_in_ram(
    sasrec_dataset, sasrec_test_data,
    kg_dataset, kg_model,
    dtype=np.float32,     # đổi np.float16 nếu thiếu RAM
    show_progress=True
)

print("[INFO] KG matrix in RAM:", kg_matrix.shape, kg_matrix.dtype)
print("[INFO] Rows (users/sequences):", len(row_uid_tokens))
print("[INFO] Common items:", len(SEQ_POS), " / SEQ items:", sasrec_dataset.item_num)


In [ ]:
import numpy as np
import torch, math
from tqdm import tqdm
from recbole.data.interaction import Interaction

# ---------- Softmax normalize ----------
def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    m = np.max(x)
    e = np.exp(x - m)
    s = e.sum()
    if s == 0:
        return np.full_like(x, 1.0 / len(x))
    return e / s

# ---------- Cấu hình ----------
assert 'kg_matrix' in globals(), "Thiếu biến kg_matrix (điểm KG trong RAM)."
sasrec_model.eval()
device = sasrec_device

# K và alpha bạn đang dùng
K_list  = [10]
alphas  = [round(a, 1) for a in np.arange(0.0, 1.0, 0.1)]
maxK = max(K_list)

# Hình dạng ma trận KG (phải khớp số cột với không gian item của SEQ)
num_rows, n_items_seq = kg_matrix.shape
if kg_matrix.dtype != np.float32:
    kg_matrix = kg_matrix.astype(np.float32, copy=False)  # đảm bảo float32 (nhanh & gọn)

# ---------- Bộ đếm metric (LS) ----------
total = 0
metrics = {a: {k: {"hits": 0, "mrr": 0.0, "ndcg": 0.0} for k in K_list} for a in alphas}
row_idx = 0

with torch.no_grad():
    for batch in tqdm(sasrec_test_data, desc="Fusion (RAM) + Evaluate"):
        # --- Ground truth (LS) an toàn ---
        b1 = batch[1] if len(batch) > 1 else None
        if (b1 is not None) and ('item_id' in b1):
            gts = b1['item_id'].cpu().tolist()
        elif 'item_id' in batch[0]:
            # một số cấu hình để GT ở batch[0]
            gts = batch[0]['item_id'].cpu().tolist()
        else:
            # không tìm thấy GT → bỏ qua batch
            continue
        bsz = len(gts)

        # --- Interaction tối thiểu cho SASRec (tránh field thừa) ---
        fields = {
            'user_id': batch[0]['user_id'],
            'item_id_list': batch[0]['item_id_list']
        }
        if 'item_length' in batch[0]:
            fields['item_length'] = batch[0]['item_length']
        inter_seq = Interaction({k: v.clone() for k, v in fields.items()})

        # --- Seq scores theo batch (GPU→CPU fallback nếu cần) ---
        try:
            s_scores = sasrec_model.full_sort_predict(inter_seq.to(device)).detach().cpu().numpy()
        except RuntimeError as e:
            print("[WARN] full_sort_predict(GPU) fail, fallback CPU:", str(e)[:120])
            m_cpu = sasrec_model.to('cpu')
            s_scores = m_cpu.full_sort_predict(inter_seq.to('cpu')).detach().cpu().numpy()
            sasrec_model.to(device)

        # --- Bảo đảm còn đủ hàng KG để so khớp ---
        if row_idx + bsz > num_rows:
            # cắt cho vừa (tránh out-of-range); in cảnh báo
            usable = num_rows - row_idx
            if usable <= 0:
                break
            print(f"[WARN] Cắt batch cuối: cần {bsz}, chỉ còn {usable} hàng KG.")
            gts = gts[:usable]
            s_scores = s_scores[:usable]
            bsz = usable

        # --- Xử lý từng hàng ---
        for b in range(bsz):
            s_norm = softmax_np(s_scores[b])
            k_norm = softmax_np(kg_matrix[row_idx])
            gt = gts[b]
            total += 1

            for a in alphas:
                fused = (1.0 - a) * s_norm + a * k_norm

                # Top-maxK (argpartition) rồi sort trong top
                idx_part = np.argpartition(-fused, maxK - 1)[:maxK]
                idx_sorted = idx_part[np.argsort(-fused[idx_part])]

                # vị trí GT trong top-maxK (nếu có)
                pos = np.where(idx_sorted == gt)[0]
                if pos.size > 0:
                    rank1 = int(pos[0]) + 1  # 1-based
                    inv_log = 1.0 / math.log2(rank1 + 1)
                    inv_rank = 1.0 / rank1
                    for K in K_list:
                        if rank1 <= K:
                            d = metrics[a][K]
                            d["hits"] += 1
                            d["mrr"]  += inv_rank
                            d["ndcg"] += inv_log

            row_idx += 1

# ---------- In kết quả ----------
print(f"\n[INFO] Tổng mẫu đánh giá: {total}  |  Rows đã dùng: {row_idx}/{num_rows}")
for a in alphas:
    for K in K_list:
        d = metrics[a][K]
        hit = d["hits"]; mrr_sum = d["mrr"]; ndcg_sum = d["ndcg"]
        recall = hit / total if total else 0.0                 # LS: Hit@K == Recall@K
        precision = hit / (total * K) if total else 0.0        # LS precision
        mrr = mrr_sum / total if total else 0.0
        ndcg = ndcg_sum / total if total else 0.0
        print(f"[alpha={a:.1f} | K={K:>2}] "
              f"Recall={recall:.5f}  Precision={precision:.5f}  NDCG={ndcg:.5f}  MRR={mrr:.5f}")

if row_idx != num_rows:
    print(f"[WARN] Số hàng KG không khớp: đã dùng {row_idx}, còn dư {num_rows - row_idx}. "
          "Đảm bảo thứ tự duyệt khi build kg_matrix khớp test_loader.")


## BERT4Rec

In [ ]:
bert4rec_config = {
    'data_path': 'D:\Personal\RecBole',
    'seed': 2020,
    'reproducibility': True,
    'device': 'cuda',

    # Training
    'epochs': 35,
    'train_batch_size': 256,
    'eval_batch_size': 512,
    'learning_rate': 0.001,
    'loss_type': 'CE',
    'train_neg_sample_args': None,  # Required for CE loss

    'eval_step': 2,
    'stopping_step': 5,
    'validation_metric': 'Recall@10',
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'
    },
    'log_wandb': False,

    # Data fields
    'field_separator': '\t',
    'seq_separator': ' ',

    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',
    'NEG_PREFIX': 'neg_',

    'alias_of_user_id': ['user_id'],
    'alias_of_item_id': ['item_id'],

    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp']
    },

    # Filtering
    'normalize_all': False,

    # Sequence-specific
    'mask_ratio': 0.2,
    'dropout_prob': 0.2,
    'MAX_ITEM_LIST_LENGTH': 40,
    'ITEM_LIST_LENGTH_FIELD': 'item_length',
    'LIST_SUFFIX': '_list'
}


In [ ]:
config = Config(model='BERT4Rec', dataset='Amazon_Books_filtered', config_dict=bert4rec_config)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
device = config['device']
model = BERT4Rec(config, dataset).to(device)

trainer = Trainer(config, model)
print("[DEBUG] Bắt đầu huấn luyện")
trainer.fit(train_data, valid_data)
res = trainer.evaluate(test_data)
print(res)
torch.save(model.state_dict(), 'BERT4Rec_AB_filtered_ls.pth')

del model
gc.collect(); torch.cuda.empty_cache()

In [ ]:
config = Config(model='BERT4Rec', dataset='Amazon_Books_filtered', config_dict=bert4rec_config)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
device = config['device']
model = BERT4Rec(config, dataset).to(device)
model.load_state_dict(torch.load('BERT4Rec_AB_filtered_ls.pth'))

# evaluate seq_model
for k in [5, 10, 20, 50]:
    model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in test_data:
        scores = model.full_sort_predict(batch[0].to(device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
import numpy as np
import torch, math
from tqdm import tqdm
from recbole.data.interaction import Interaction

# ---------- Softmax normalize ----------
def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    m = np.max(x)
    e = np.exp(x - m)
    s = e.sum()
    if s == 0:
        return np.full_like(x, 1.0 / len(x))
    return e / s

# ---------- Cấu hình ----------
assert 'kg_matrix' in globals(), "Thiếu biến kg_matrix (điểm KG trong RAM)."
model.eval()
device = model.device

# K và alpha bạn đang dùng
K_list  = [10]
alphas  = [round(a, 1) for a in np.arange(0.0, 1.0, 0.1)]
maxK = max(K_list)

# Hình dạng ma trận KG (phải khớp số cột với không gian item của SEQ)
num_rows, n_items_seq = kg_matrix.shape
if kg_matrix.dtype != np.float32:
    kg_matrix = kg_matrix.astype(np.float32, copy=False)  # đảm bảo float32 (nhanh & gọn)

# ---------- Bộ đếm metric (LS) ----------
total = 0
metrics = {a: {k: {"hits": 0, "mrr": 0.0, "ndcg": 0.0} for k in K_list} for a in alphas}
row_idx = 0

with torch.no_grad():
    for batch in tqdm(sasrec_test_data, desc="Fusion (RAM) + Evaluate"):
        # --- Ground truth (LS) an toàn ---
        b1 = batch[1] if len(batch) > 1 else None
        if (b1 is not None) and ('item_id' in b1):
            gts = b1['item_id'].cpu().tolist()
        elif 'item_id' in batch[0]:
            # một số cấu hình để GT ở batch[0]
            gts = batch[0]['item_id'].cpu().tolist()
        else:
            # không tìm thấy GT → bỏ qua batch
            continue
        bsz = len(gts)

        fields = {
            'user_id': batch[0]['user_id'],
            'item_id_list': batch[0]['item_id_list']
        }
        if 'item_length' in batch[0]:
            fields['item_length'] = batch[0]['item_length']
        inter_seq = Interaction({k: v.clone() for k, v in fields.items()})

        # --- Seq scores theo batch (GPU→CPU fallback nếu cần) ---
        try:
            s_scores = model.full_sort_predict(inter_seq.to(device)).detach().cpu().numpy()
        except RuntimeError as e:
            print("[WARN] full_sort_predict(GPU) fail, fallback CPU:", str(e)[:120])
            m_cpu = model.to('cpu')
            s_scores = m_cpu.full_sort_predict(inter_seq.to('cpu')).detach().cpu().numpy()
            model.to(device)

        # --- Bảo đảm còn đủ hàng KG để so khớp ---
        if row_idx + bsz > num_rows:
            # cắt cho vừa (tránh out-of-range); in cảnh báo
            usable = num_rows - row_idx
            if usable <= 0:
                break
            print(f"[WARN] Cắt batch cuối: cần {bsz}, chỉ còn {usable} hàng KG.")
            gts = gts[:usable]
            s_scores = s_scores[:usable]
            bsz = usable

        # --- Xử lý từng hàng ---
        for b in range(bsz):
            s_norm = softmax_np(s_scores[b])
            k_norm = softmax_np(kg_matrix[row_idx])
            gt = gts[b]
            total += 1

            for a in alphas:
                fused = (1.0 - a) * s_norm + a * k_norm

                # Top-maxK (argpartition) rồi sort trong top
                idx_part = np.argpartition(-fused, maxK - 1)[:maxK]
                idx_sorted = idx_part[np.argsort(-fused[idx_part])]

                # vị trí GT trong top-maxK (nếu có)
                pos = np.where(idx_sorted == gt)[0]
                if pos.size > 0:
                    rank1 = int(pos[0]) + 1  # 1-based
                    inv_log = 1.0 / math.log2(rank1 + 1)
                    inv_rank = 1.0 / rank1
                    for K in K_list:
                        if rank1 <= K:
                            d = metrics[a][K]
                            d["hits"] += 1
                            d["mrr"]  += inv_rank
                            d["ndcg"] += inv_log

            row_idx += 1

# ---------- In kết quả ----------
print(f"\n[INFO] Tổng mẫu đánh giá: {total}  |  Rows đã dùng: {row_idx}/{num_rows}")
for a in alphas:
    for K in K_list:
        d = metrics[a][K]
        hit = d["hits"]; mrr_sum = d["mrr"]; ndcg_sum = d["ndcg"]
        recall = hit / total if total else 0.0                 # LS: Hit@K == Recall@K
        precision = hit / (total * K) if total else 0.0        # LS precision
        mrr = mrr_sum / total if total else 0.0
        ndcg = ndcg_sum / total if total else 0.0
        print(f"[alpha={a:.1f} | K={K:>2}] "
              f"Recall={recall:.5f}  Precision={precision:.5f}  NDCG={ndcg:.5f}  MRR={mrr:.5f}")

if row_idx != num_rows:
    print(f"[WARN] Số hàng KG không khớp: đã dùng {row_idx}, còn dư {num_rows - row_idx}. "
          "Đảm bảo thứ tự duyệt khi build kg_matrix khớp test_loader.")


## FEARec

In [ ]:
fearec_config = {
    'data_path': 'D:\Personal\RecBole',
    'seed': 2020,
    'reproducibility': True,
    'device': 'cuda',

    'epochs': 35,
    'train_batch_size': 256,
    'eval_batch_size': 512,
    'learning_rate': 1e-3,

    'loss_type': 'CE',                     
    'train_neg_sample_args': None,        

    'eval_step': 2,
    'stopping_step': 5,
    'validation_metric': 'Recall@10',
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'
    },
    'log_wandb': False,

    'field_separator': '\t',
    'seq_separator': ' ',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'RATING_FIELD': 'rating',
    'TIME_FIELD': 'timestamp',
    'LABEL_FIELD': 'label',
    'NEG_PREFIX': 'neg_',
    'alias_of_user_id': ['user_id'],
    'alias_of_item_id': ['item_id'],
    'load_col': {
        'inter': ['user_id', 'item_id', 'rating', 'timestamp']
    },
    'normalize_all': False,

    'MAX_ITEM_LIST_LENGTH': 40,
    'ITEM_LIST_LENGTH_FIELD': 'item_length',
    'LIST_SUFFIX': '_list',

    'hidden_size': 64,           
    'inner_size': 256,            
    'n_layers': 2,
    'n_heads': 2,
    'hidden_dropout_prob': 0.5,
    'attn_dropout_prob': 0.5,
    'hidden_act': 'gelu',          
    'layer_norm_eps': 1e-12,
    'initializer_range': 0.02,

    'global_ratio': 1.0,          
    'dual_domain': False,          
    'std': False,                 
    'fredom': False,               
    'spatial_ratio': 0.0,          
    'topk_factor': 1,              
    'lmd': 0.1,                    
    'lmd_sem': 0.1,                
    'fredom_type': None,           
}


In [ ]:
config = Config(model='FEARec', dataset='Amazon_Books_filtered', config_dict=fearec_config)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
device = config['device']
model = FEARec(config, dataset).to(device)

trainer = Trainer(config, model)
print("[DEBUG] Bắt đầu huấn luyện")
trainer.fit(train_data, valid_data)
res = trainer.evaluate(test_data)
print(res)
torch.save(model.state_dict(), 'FEARec_AB_filtered_ls.pth')

del model
gc.collect(); torch.cuda.empty_cache()

In [ ]:
config = Config(model='FEARec', dataset='Amazon_Books_filtered', config_dict=fearec_config)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
device = config['device']
model = FEARec(config, dataset).to(device)
model.load_state_dict(torch.load('FEARec_AB_filtered_ls.pth'))

# evaluate seq_model
for k in [5, 10, 20, 50]:
    model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in test_data:
        scores = model.full_sort_predict(batch[0].to(device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
import numpy as np
import torch, math
from tqdm import tqdm
from recbole.data.interaction import Interaction

# ---------- Softmax normalize ----------
def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    m = np.max(x)
    e = np.exp(x - m)
    s = e.sum()
    if s == 0:
        return np.full_like(x, 1.0 / len(x))
    return e / s

# ---------- Cấu hình ----------
assert 'kg_matrix' in globals(), "Thiếu biến kg_matrix (điểm KG trong RAM)."
model.eval()
device = model.device

# K và alpha bạn đang dùng
K_list  = [10]
alphas  = [round(a, 1) for a in np.arange(0.0, 1.0, 0.1)]
maxK = max(K_list)

# Hình dạng ma trận KG (phải khớp số cột với không gian item của SEQ)
num_rows, n_items_seq = kg_matrix.shape
if kg_matrix.dtype != np.float32:
    kg_matrix = kg_matrix.astype(np.float32, copy=False)  # đảm bảo float32 (nhanh & gọn)

# ---------- Bộ đếm metric (LS) ----------
total = 0
metrics = {a: {k: {"hits": 0, "mrr": 0.0, "ndcg": 0.0} for k in K_list} for a in alphas}
row_idx = 0

with torch.no_grad():
    for batch in tqdm(sasrec_test_data, desc="Fusion (RAM) + Evaluate"):
        # --- Ground truth (LS) an toàn ---
        b1 = batch[1] if len(batch) > 1 else None
        if (b1 is not None) and ('item_id' in b1):
            gts = b1['item_id'].cpu().tolist()
        elif 'item_id' in batch[0]:
            # một số cấu hình để GT ở batch[0]
            gts = batch[0]['item_id'].cpu().tolist()
        else:
            # không tìm thấy GT → bỏ qua batch
            continue
        bsz = len(gts)

        fields = {
            'user_id': batch[0]['user_id'],
            'item_id_list': batch[0]['item_id_list']
        }
        if 'item_length' in batch[0]:
            fields['item_length'] = batch[0]['item_length']
        inter_seq = Interaction({k: v.clone() for k, v in fields.items()})

        # --- Seq scores theo batch (GPU→CPU fallback nếu cần) ---
        try:
            s_scores = model.full_sort_predict(inter_seq.to(device)).detach().cpu().numpy()
        except RuntimeError as e:
            print("[WARN] full_sort_predict(GPU) fail, fallback CPU:", str(e)[:120])
            m_cpu = model.to('cpu')
            s_scores = m_cpu.full_sort_predict(inter_seq.to('cpu')).detach().cpu().numpy()
            model.to(device)

        # --- Bảo đảm còn đủ hàng KG để so khớp ---
        if row_idx + bsz > num_rows:
            # cắt cho vừa (tránh out-of-range); in cảnh báo
            usable = num_rows - row_idx
            if usable <= 0:
                break
            print(f"[WARN] Cắt batch cuối: cần {bsz}, chỉ còn {usable} hàng KG.")
            gts = gts[:usable]
            s_scores = s_scores[:usable]
            bsz = usable

        # --- Xử lý từng hàng ---
        for b in range(bsz):
            s_norm = softmax_np(s_scores[b])
            k_norm = softmax_np(kg_matrix[row_idx])
            gt = gts[b]
            total += 1

            for a in alphas:
                fused = (1.0 - a) * s_norm + a * k_norm

                # Top-maxK (argpartition) rồi sort trong top
                idx_part = np.argpartition(-fused, maxK - 1)[:maxK]
                idx_sorted = idx_part[np.argsort(-fused[idx_part])]

                # vị trí GT trong top-maxK (nếu có)
                pos = np.where(idx_sorted == gt)[0]
                if pos.size > 0:
                    rank1 = int(pos[0]) + 1  # 1-based
                    inv_log = 1.0 / math.log2(rank1 + 1)
                    inv_rank = 1.0 / rank1
                    for K in K_list:
                        if rank1 <= K:
                            d = metrics[a][K]
                            d["hits"] += 1
                            d["mrr"]  += inv_rank
                            d["ndcg"] += inv_log

            row_idx += 1

# ---------- In kết quả ----------
print(f"\n[INFO] Tổng mẫu đánh giá: {total}  |  Rows đã dùng: {row_idx}/{num_rows}")
for a in alphas:
    for K in K_list:
        d = metrics[a][K]
        hit = d["hits"]; mrr_sum = d["mrr"]; ndcg_sum = d["ndcg"]
        recall = hit / total if total else 0.0                 # LS: Hit@K == Recall@K
        precision = hit / (total * K) if total else 0.0        # LS precision
        mrr = mrr_sum / total if total else 0.0
        ndcg = ndcg_sum / total if total else 0.0
        print(f"[alpha={a:.1f} | K={K:>2}] "
              f"Recall={recall:.5f}  Precision={precision:.5f}  NDCG={ndcg:.5f}  MRR={mrr:.5f}")

if row_idx != num_rows:
    print(f"[WARN] Số hàng KG không khớp: đã dùng {row_idx}, còn dư {num_rows - row_idx}. "
          "Đảm bảo thứ tự duyệt khi build kg_matrix khớp test_loader.")


# LastFM_1B

## SASRec

In [ ]:
sasrec_config, sasrec_model, sasrec_dataset, sasrec_train_data, sasrec_valid_data, sasrec_test_data = load_data_and_model(
    model_file='SASRec_LastFM_filtered.pth'
)

In [ ]:
device = sasrec_config['device']

# evaluate seq_model
for k in [5, 10, 20, 50]:
    sasrec_model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in sasrec_test_data:
        scores = sasrec_model.full_sort_predict(batch[0].to(device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
import torch

ckpt_path = "KGIN_LastFM_filtered.pth"
ckpt = torch.load(ckpt_path, map_location='cpu')

cfg = ckpt.get('config', {})
cfg['data_path'] = "./datasets/LastFM_filtered"     # PATH mới trên máy bạn
cfg['dataset']  = "LastFM_filtered"                   # phải khớp prefix file dữ liệu

ckpt['config'] = cfg
fixed_path = "D:\Personal\RecBole\KGIN_LastFM_filtered_fixed.pth"
torch.save(ckpt, fixed_path)
print('Saved:', fixed_path)

kgin_config, kgin_model, kgin_dataset, kgin_train_data, kgin_valid_data, kgin_test_data = load_data_and_model(
    model_file='KGIN_LastFM_filtered_fixed.pth'
)

In [ ]:
import numpy as np
import torch
from tqdm import tqdm
from recbole.data.interaction import Interaction

def build_kg_matrix_in_ram(
    sasrec_dataset, sasrec_test_data,
    kg_dataset, kg_model,
    dtype=np.float32,           # đổi sang np.float16 nếu cần tiết kiệm RAM
    show_progress=True
):
    """
    Trả về:
      - kg_matrix: np.ndarray shape [num_rows, n_items_seq], dtype=dtype
      - row_uid_tokens: list[str] thứ tự user_token tương ứng từng hàng
      - seq_pos: np.ndarray[int64] các cột SEQ có tương ứng trong KG
      - kg_ids : np.ndarray[int64] id item tương ứng bên KG (cùng chiều với seq_pos)
    """

    kg_model.eval()
    kg_device = next(kg_model.parameters()).device

    # === Không gian item SEQ và map sang KG ===
    n_items_seq = sasrec_dataset.item_num
    seq_tokens = [sasrec_dataset.id2token('item_id', i) for i in range(n_items_seq)]
    kg_tok2id_item = kg_dataset.field2token_id['item_id']  # token->id (KG)
    kg_idx_for_seq = np.array([kg_tok2id_item.get(t, -1) for t in seq_tokens], dtype=np.int64)

    # Giới hạn theo kích thước embedding hiện có của KG (an toàn CUDA)
    ue = getattr(kg_model, "user_embedding", None) or getattr(kg_model, "entity_embedding", None)
    ie = getattr(kg_model, "item_embedding", None) or getattr(kg_model, "relation_embedding", None)
    MAX_U = None if ue is None else ue.num_embeddings
    MAX_I = None if ie is None else ie.num_embeddings

    valid_mask = kg_idx_for_seq >= 0
    if MAX_I is not None:
        valid_mask &= (kg_idx_for_seq < MAX_I)

    seq_pos = np.flatnonzero(valid_mask)     # cột trong SEQ có tương ứng ở KG
    kg_ids  = kg_idx_for_seq[seq_pos]        # item_id tương ứng bên KG

    if len(seq_pos) == 0:
        raise RuntimeError("Không có item chung hợp lệ giữa SEQ và KG (seq_pos rỗng).")

    # ====== Duyệt test loader & ghép thành ma trận trong RAM ======
    rows = []
    row_uid_tokens = []

    it = tqdm(sasrec_test_data, desc="KG → RAM (per-row)", disable=not show_progress)
    with torch.no_grad():
        for batch in it:
            inp = batch[0]
            user_ids   = inp['user_id'].cpu().tolist()
            item_lists = inp['item_id_list'].cpu().tolist()   # index theo SEQ

            for i, uid in enumerate(user_ids):
                # Hàng kết quả (không gian SEQ), mặc định 0.0 cho item không có trong KG
                row = np.zeros(n_items_seq, dtype=dtype)

                uid_tok = sasrec_dataset.id2token('user_id', uid)
                row_uid_tokens.append(uid_tok)

                # Tính điểm KG chỉ trên item chung
                sub = None
                if (uid_tok in kg_dataset.field2token_id['user_id']) and (len(seq_pos) > 0):
                    kg_uid = kg_dataset.token2id('user_id', uid_tok)
                    if (MAX_U is None) or (kg_uid < MAX_U):
                        inter = Interaction({
                            'user_id': torch.full((len(kg_ids),), kg_uid, dtype=torch.long),
                            'item_id': torch.as_tensor(kg_ids, dtype=torch.long),
                        }).to(kg_device)

                        try:
                            out = kg_model.predict(inter)
                        except AttributeError:
                            out = kg_model.forward(inter)

                        sub = out.detach().cpu().numpy().reshape(-1).astype(dtype)  # len == len(kg_ids)
                        row[seq_pos] = sub

                # Gán min-score cho item đã tương tác (index theo SEQ), loại pad=0 nếu có
                if sub is not None and sub.size > 0:
                    min_score = dtype(sub.min())
                else:
                    min_score = dtype(0.0)

                seen_idx = np.fromiter((x for x in set(item_lists[i]) if 0 < x < n_items_seq), dtype=np.int64)
                if seen_idx.size > 0:
                    row[seen_idx] = min_score

                rows.append(row)

    # Ghép tất cả hàng thành 1 ma trận trong RAM
    kg_matrix = np.stack(rows, axis=0)
    return kg_matrix, row_uid_tokens, seq_pos, kg_ids

# ==== Ví dụ gọi hàm (giữ trong RAM) ====
# CẢNH BÁO RAM: cân nhắc đổi dtype=np.float16 nếu n_items_seq lớn
kg_matrix, row_uid_tokens, SEQ_POS, KG_IDS = build_kg_matrix_in_ram(
    sasrec_dataset, sasrec_test_data,
    kgin_dataset, kgin_model,
    dtype=np.float32,     # đổi np.float16 nếu thiếu RAM
    show_progress=True
)

print("[INFO] KG matrix in RAM:", kg_matrix.shape, kg_matrix.dtype)
print("[INFO] Rows (users/sequences):", len(row_uid_tokens))
print("[INFO] Common items:", len(SEQ_POS), " / SEQ items:", sasrec_dataset.item_num)


In [ ]:
import numpy as np
import torch, math
from tqdm import tqdm
from recbole.data.interaction import Interaction

# ---------- Softmax normalize ----------
def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    m = np.max(x)
    e = np.exp(x - m)
    s = e.sum()
    if s == 0:
        return np.full_like(x, 1.0 / len(x))
    return e / s

# ---------- Cấu hình ----------
assert 'kg_matrix' in globals(), "Thiếu biến kg_matrix (điểm KG trong RAM)."
sasrec_model.eval()
device = sasrec_config['device']

# K và alpha bạn đang dùng
K_list  = [10]
alphas  = [round(a, 1) for a in np.arange(0.0, 1.0, 0.1)]
maxK = max(K_list)

# Hình dạng ma trận KG (phải khớp số cột với không gian item của SEQ)
num_rows, n_items_seq = kg_matrix.shape
if kg_matrix.dtype != np.float32:
    kg_matrix = kg_matrix.astype(np.float32, copy=False)  # đảm bảo float32 (nhanh & gọn)

# ---------- Bộ đếm metric (LS) ----------
total = 0
metrics = {a: {k: {"hits": 0, "mrr": 0.0, "ndcg": 0.0} for k in K_list} for a in alphas}
row_idx = 0

with torch.no_grad():
    for batch in tqdm(sasrec_test_data, desc="Fusion (RAM) + Evaluate"):
        # --- Ground truth (LS) an toàn ---
        b1 = batch[1] if len(batch) > 1 else None
        if (b1 is not None) and ('item_id' in b1):
            gts = b1['item_id'].cpu().tolist()
        elif 'item_id' in batch[0]:
            # một số cấu hình để GT ở batch[0]
            gts = batch[0]['item_id'].cpu().tolist()
        else:
            # không tìm thấy GT → bỏ qua batch
            continue
        bsz = len(gts)

        # --- Interaction tối thiểu cho SASRec (tránh field thừa) ---
        fields = {
            'user_id': batch[0]['user_id'],
            'item_id_list': batch[0]['item_id_list']
        }
        if 'item_length' in batch[0]:
            fields['item_length'] = batch[0]['item_length']
        inter_seq = Interaction({k: v.clone() for k, v in fields.items()})

        # --- Seq scores theo batch (GPU→CPU fallback nếu cần) ---
        try:
            s_scores = sasrec_model.full_sort_predict(inter_seq.to(device)).detach().cpu().numpy()
        except RuntimeError as e:
            print("[WARN] full_sort_predict(GPU) fail, fallback CPU:", str(e)[:120])
            m_cpu = sasrec_model.to('cpu')
            s_scores = m_cpu.full_sort_predict(inter_seq.to('cpu')).detach().cpu().numpy()
            sasrec_model.to(device)

        # --- Bảo đảm còn đủ hàng KG để so khớp ---
        if row_idx + bsz > num_rows:
            # cắt cho vừa (tránh out-of-range); in cảnh báo
            usable = num_rows - row_idx
            if usable <= 0:
                break
            print(f"[WARN] Cắt batch cuối: cần {bsz}, chỉ còn {usable} hàng KG.")
            gts = gts[:usable]
            s_scores = s_scores[:usable]
            bsz = usable

        # --- Xử lý từng hàng ---
        for b in range(bsz):
            s_norm = softmax_np(s_scores[b])
            k_norm = softmax_np(kg_matrix[row_idx])
            gt = gts[b]
            total += 1

            for a in alphas:
                fused = (1.0 - a) * s_norm + a * k_norm

                # Top-maxK (argpartition) rồi sort trong top
                idx_part = np.argpartition(-fused, maxK - 1)[:maxK]
                idx_sorted = idx_part[np.argsort(-fused[idx_part])]

                # vị trí GT trong top-maxK (nếu có)
                pos = np.where(idx_sorted == gt)[0]
                if pos.size > 0:
                    rank1 = int(pos[0]) + 1  # 1-based
                    inv_log = 1.0 / math.log2(rank1 + 1)
                    inv_rank = 1.0 / rank1
                    for K in K_list:
                        if rank1 <= K:
                            d = metrics[a][K]
                            d["hits"] += 1
                            d["mrr"]  += inv_rank
                            d["ndcg"] += inv_log

            row_idx += 1

# ---------- In kết quả ----------
print(f"\n[INFO] Tổng mẫu đánh giá: {total}  |  Rows đã dùng: {row_idx}/{num_rows}")
for a in alphas:
    for K in K_list:
        d = metrics[a][K]
        hit = d["hits"]; mrr_sum = d["mrr"]; ndcg_sum = d["ndcg"]
        recall = hit / total if total else 0.0                 # LS: Hit@K == Recall@K
        precision = hit / (total * K) if total else 0.0        # LS precision
        mrr = mrr_sum / total if total else 0.0
        ndcg = ndcg_sum / total if total else 0.0
        print(f"[alpha={a:.1f} | K={K:>2}] "
              f"Recall={recall:.5f}  Precision={precision:.5f}  NDCG={ndcg:.5f}  MRR={mrr:.5f}")

if row_idx != num_rows:
    print(f"[WARN] Số hàng KG không khớp: đã dùng {row_idx}, còn dư {num_rows - row_idx}. "
          "Đảm bảo thứ tự duyệt khi build kg_matrix khớp test_loader.")


## BERT4Rec

In [ ]:
import torch

ckpt_path = "BERT4Rec_LastFM_filtered.pth"
ckpt = torch.load(ckpt_path, map_location='cpu')

cfg = ckpt.get('config', {})
cfg['data_path'] = "./datasets/LastFM_filtered"     # PATH mới trên máy bạn
cfg['dataset']  = "LastFM_filtered"                   # phải khớp prefix file dữ liệu

ckpt['config'] = cfg
fixed_path = "D:\Personal\RecBole\BERT4Rec_LastFM_filtered_fixed.pth"
torch.save(ckpt, fixed_path)
print('Saved:', fixed_path)

bert4rec_config, bert4rec_model, bert4rec_dataset, bert4rec_train_data, bert4rec_valid_data, bert4rec_test_data = load_data_and_model(
    model_file='BERT4Rec_LastFM_filtered_fixed.pth'
)

device = bert4rec_config['device']

# evaluate seq_model
for k in [5, 10, 20, 50]:
    bert4rec_model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in bert4rec_test_data:
        scores = bert4rec_model.full_sort_predict(batch[0].to(device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
import numpy as np
import torch, math
from tqdm import tqdm
from recbole.data.interaction import Interaction

def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    m = np.max(x)
    e = np.exp(x - m)
    s = e.sum()
    if s == 0:
        return np.full_like(x, 1.0 / len(x))
    return e / s

assert 'kg_matrix' in globals(), "Thiếu biến kg_matrix (điểm KG trong RAM)."
bert4rec_model.eval()
device = bert4rec_config['device']

K_list  = [10]
alphas  = [round(a, 1) for a in np.arange(0.0, 1.0, 0.1)]
maxK = max(K_list)

num_rows, n_items_seq = kg_matrix.shape
if kg_matrix.dtype != np.float32:
    kg_matrix = kg_matrix.astype(np.float32, copy=False)  

total = 0
metrics = {a: {k: {"hits": 0, "mrr": 0.0, "ndcg": 0.0} for k in K_list} for a in alphas}
row_idx = 0

with torch.no_grad():
    for batch in tqdm(sasrec_test_data, desc="Fusion (RAM) + Evaluate"):
        # --- Ground truth (LS) an toàn ---
        b1 = batch[1] if len(batch) > 1 else None
        if (b1 is not None) and ('item_id' in b1):
            gts = b1['item_id'].cpu().tolist()
        elif 'item_id' in batch[0]:
            # một số cấu hình để GT ở batch[0]
            gts = batch[0]['item_id'].cpu().tolist()
        else:
            # không tìm thấy GT → bỏ qua batch
            continue
        bsz = len(gts)

        # --- Interaction tối thiểu cho SASRec (tránh field thừa) ---
        fields = {
            'user_id': batch[0]['user_id'],
            'item_id_list': batch[0]['item_id_list']
        }
        if 'item_length' in batch[0]:
            fields['item_length'] = batch[0]['item_length']
        inter_seq = Interaction({k: v.clone() for k, v in fields.items()})

        # --- Seq scores theo batch (GPU→CPU fallback nếu cần) ---
        try:
            s_scores = bert4rec_model.full_sort_predict(inter_seq.to(device)).detach().cpu().numpy()
        except RuntimeError as e:
            print("[WARN] full_sort_predict(GPU) fail, fallback CPU:", str(e)[:120])
            m_cpu = bert4rec_model.to('cpu')
            s_scores = m_cpu.full_sort_predict(inter_seq.to('cpu')).detach().cpu().numpy()
            bert4rec_model.to(device)

        # --- Bảo đảm còn đủ hàng KG để so khớp ---
        if row_idx + bsz > num_rows:
            # cắt cho vừa (tránh out-of-range); in cảnh báo
            usable = num_rows - row_idx
            if usable <= 0:
                break
            print(f"[WARN] Cắt batch cuối: cần {bsz}, chỉ còn {usable} hàng KG.")
            gts = gts[:usable]
            s_scores = s_scores[:usable]
            bsz = usable

        # --- Xử lý từng hàng ---
        for b in range(bsz):
            s_norm = softmax_np(s_scores[b])
            k_norm = softmax_np(kg_matrix[row_idx])
            gt = gts[b]
            total += 1

            for a in alphas:
                fused = (1.0 - a) * s_norm + a * k_norm

                # Top-maxK (argpartition) rồi sort trong top
                idx_part = np.argpartition(-fused, maxK - 1)[:maxK]
                idx_sorted = idx_part[np.argsort(-fused[idx_part])]

                # vị trí GT trong top-maxK (nếu có)
                pos = np.where(idx_sorted == gt)[0]
                if pos.size > 0:
                    rank1 = int(pos[0]) + 1  # 1-based
                    inv_log = 1.0 / math.log2(rank1 + 1)
                    inv_rank = 1.0 / rank1
                    for K in K_list:
                        if rank1 <= K:
                            d = metrics[a][K]
                            d["hits"] += 1
                            d["mrr"]  += inv_rank
                            d["ndcg"] += inv_log

            row_idx += 1

# ---------- In kết quả ----------
print(f"\n[INFO] Tổng mẫu đánh giá: {total}  |  Rows đã dùng: {row_idx}/{num_rows}")
for a in alphas:
    for K in K_list:
        d = metrics[a][K]
        hit = d["hits"]; mrr_sum = d["mrr"]; ndcg_sum = d["ndcg"]
        recall = hit / total if total else 0.0                 # LS: Hit@K == Recall@K
        precision = hit / (total * K) if total else 0.0        # LS precision
        mrr = mrr_sum / total if total else 0.0
        ndcg = ndcg_sum / total if total else 0.0
        print(f"[alpha={a:.1f} | K={K:>2}] "
              f"Recall={recall:.5f}  Precision={precision:.5f}  NDCG={ndcg:.5f}  MRR={mrr:.5f}")

if row_idx != num_rows:
    print(f"[WARN] Số hàng KG không khớp: đã dùng {row_idx}, còn dư {num_rows - row_idx}. "
          "Đảm bảo thứ tự duyệt khi build kg_matrix khớp test_loader.")


## FEARec

In [ ]:
import torch

ckpt_path = "FEARec_LastFM_filtered.pth"
ckpt = torch.load(ckpt_path, map_location='cpu')

cfg = ckpt.get('config', {})
cfg['data_path'] = "./datasets/LastFM_filtered"     # PATH mới trên máy bạn
cfg['dataset']  = "LastFM_filtered"                   # phải khớp prefix file dữ liệu

ckpt['config'] = cfg
fixed_path = "D:\Personal\RecBole\FEA4Rec_LastFM_filtered_fixed.pth"
torch.save(ckpt, fixed_path)
print('Saved:', fixed_path)

fearec_config, fearec_model, fearec_dataset, fearec_train_data, fearec_valid_data, fearec_test_data = load_data_and_model(
    model_file='FEA4Rec_LastFM_filtered_fixed.pth'
)

device = fearec_config['device']

# evaluate seq_model
for k in [5, 10, 20, 50]:
    fearec_model.eval()
    all_ranks = []
    all_ground_truths = []

    for batch in fearec_test_data:
        scores = fearec_model.full_sort_predict(batch[0].to(device))

        # Get top-K predicted items
        topk = scores.topk(k=k, dim=-1).indices
        all_ranks.append(topk.cpu())

        # Ground truth
        ground_truth = batch[0]['item_id']
        all_ground_truths.append(ground_truth.cpu())

    topk_items = torch.cat(all_ranks, dim=0).tolist()
    true_items = torch.cat(all_ground_truths, dim=0).tolist()

    recall = recall_at_k(topk_items, true_items, k=k)
    ndcg = mean_ndcg_at_k_ls(topk_items, true_items, k=k)
    mrr = mrr_at_k(topk_items, true_items, k=k)
    precision = precision_at_k(topk_items, true_items, k=k)

    print(f'Recall@{k}: {recall:.4f}, NDCG@{k}: {ndcg:.4f}, MRR@{k}: {mrr:.4f}, precision@{k}: {precision:.4f}')

In [ ]:
import numpy as np
import torch, math
from tqdm import tqdm
from recbole.data.interaction import Interaction

# ---------- Softmax normalize ----------
def softmax_np(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32, copy=False)
    m = np.max(x)
    e = np.exp(x - m)
    s = e.sum()
    if s == 0:
        return np.full_like(x, 1.0 / len(x))
    return e / s

# ---------- Cấu hình ----------
assert 'kg_matrix' in globals(), "Thiếu biến kg_matrix (điểm KG trong RAM)."
fearec_model.eval()
device = fearec_config['device']

# K và alpha bạn đang dùng
K_list  = [10]
alphas  = [round(a, 1) for a in np.arange(0.0, 1.0, 0.1)]
maxK = max(K_list)

# Hình dạng ma trận KG (phải khớp số cột với không gian item của SEQ)
num_rows, n_items_seq = kg_matrix.shape
if kg_matrix.dtype != np.float32:
    kg_matrix = kg_matrix.astype(np.float32, copy=False)  # đảm bảo float32 (nhanh & gọn)

# ---------- Bộ đếm metric (LS) ----------
total = 0
metrics = {a: {k: {"hits": 0, "mrr": 0.0, "ndcg": 0.0} for k in K_list} for a in alphas}
row_idx = 0

with torch.no_grad():
    for batch in tqdm(sasrec_test_data, desc="Fusion (RAM) + Evaluate"):
        # --- Ground truth (LS) an toàn ---
        b1 = batch[1] if len(batch) > 1 else None
        if (b1 is not None) and ('item_id' in b1):
            gts = b1['item_id'].cpu().tolist()
        elif 'item_id' in batch[0]:
            # một số cấu hình để GT ở batch[0]
            gts = batch[0]['item_id'].cpu().tolist()
        else:
            # không tìm thấy GT → bỏ qua batch
            continue
        bsz = len(gts)

        # --- Interaction tối thiểu cho SASRec (tránh field thừa) ---
        fields = {
            'user_id': batch[0]['user_id'],
            'item_id_list': batch[0]['item_id_list']
        }
        if 'item_length' in batch[0]:
            fields['item_length'] = batch[0]['item_length']
        inter_seq = Interaction({k: v.clone() for k, v in fields.items()})

        # --- Seq scores theo batch (GPU→CPU fallback nếu cần) ---
        try:
            s_scores = fearec_model.full_sort_predict(inter_seq.to(device)).detach().cpu().numpy()
        except RuntimeError as e:
            print("[WARN] full_sort_predict(GPU) fail, fallback CPU:", str(e)[:120])
            m_cpu = fearec_model.to('cpu')
            s_scores = m_cpu.full_sort_predict(inter_seq.to('cpu')).detach().cpu().numpy()
            fearec_model.to(device)

        # --- Bảo đảm còn đủ hàng KG để so khớp ---
        if row_idx + bsz > num_rows:
            # cắt cho vừa (tránh out-of-range); in cảnh báo
            usable = num_rows - row_idx
            if usable <= 0:
                break
            print(f"[WARN] Cắt batch cuối: cần {bsz}, chỉ còn {usable} hàng KG.")
            gts = gts[:usable]
            s_scores = s_scores[:usable]
            bsz = usable

        # --- Xử lý từng hàng ---
        for b in range(bsz):
            s_norm = softmax_np(s_scores[b])
            k_norm = softmax_np(kg_matrix[row_idx])
            gt = gts[b]
            total += 1

            for a in alphas:
                fused = (1.0 - a) * s_norm + a * k_norm

                # Top-maxK (argpartition) rồi sort trong top
                idx_part = np.argpartition(-fused, maxK - 1)[:maxK]
                idx_sorted = idx_part[np.argsort(-fused[idx_part])]

                # vị trí GT trong top-maxK (nếu có)
                pos = np.where(idx_sorted == gt)[0]
                if pos.size > 0:
                    rank1 = int(pos[0]) + 1  # 1-based
                    inv_log = 1.0 / math.log2(rank1 + 1)
                    inv_rank = 1.0 / rank1
                    for K in K_list:
                        if rank1 <= K:
                            d = metrics[a][K]
                            d["hits"] += 1
                            d["mrr"]  += inv_rank
                            d["ndcg"] += inv_log

            row_idx += 1

# ---------- In kết quả ----------
print(f"\n[INFO] Tổng mẫu đánh giá: {total}  |  Rows đã dùng: {row_idx}/{num_rows}")
for a in alphas:
    for K in K_list:
        d = metrics[a][K]
        hit = d["hits"]; mrr_sum = d["mrr"]; ndcg_sum = d["ndcg"]
        recall = hit / total if total else 0.0                 # LS: Hit@K == Recall@K
        precision = hit / (total * K) if total else 0.0        # LS precision
        mrr = mrr_sum / total if total else 0.0
        ndcg = ndcg_sum / total if total else 0.0
        print(f"[alpha={a:.1f} | K={K:>2}] "
              f"Recall={recall:.5f}  Precision={precision:.5f}  NDCG={ndcg:.5f}  MRR={mrr:.5f}")

if row_idx != num_rows:
    print(f"[WARN] Số hàng KG không khớp: đã dùng {row_idx}, còn dư {num_rows - row_idx}. "
          "Đảm bảo thứ tự duyệt khi build kg_matrix khớp test_loader.")
